In [30]:
from data_handler import return_create_statement_from_dataframe
import pandas as pd
from db_handler import create_connection,execute_query, close_connection
from data_handler import insert_statements_from_dataframe,extract_data_into_df,create_etl_watermark_table,insert_data_in_batches
from data_handler import execute_sql_script_from_config
from lookups import FileType
from prehook import execute
import prehook

db_session = create_connection()

In [31]:
prehook.execute()


V1_prehook_Create_ETL_watermark_table.sql
V1_prehook_Create_Schema.sql
_prehook_create_stg_kaggle_spotify_tracks.sql
_prehook_create_stg_kaggle_spotify_users.sql


In [32]:
execute_sql_script_from_config('SQL_Commands/Create_genre_mapping_table.sql')
df = pd.read_excel("C:\\dataproject\\genre_mapping.xlsx")
insert_statement = insert_statements_from_dataframe(df, 'musicschema', 'genre_mapping')
# insert_statements2 = insert_statements_from_dataframe(df1, 'musicschema', 'stg_kaggle_spotify_tracks')

with db_session.cursor() as cursor:
    for statement in insert_statement:
        cursor.execute(statement)
    
db_session.commit()

SQL script executed successfully.


In [33]:
execute_sql_script_from_config('SQL_Commands/V1_prehook_Create_ETL_watermark_table.sql')

execute_sql_script_from_config('SQL_Commands/V2_hook_create_dim_users_table.sql')

execute_sql_script_from_config('SQL_Commands/V3_hook_create_dim_genre.sql')

execute_sql_script_from_config('SQL_Commands/V4_hook_create_dim_songs.sql')

execute_sql_script_from_config('SQL_Commands/V5_hook_create_fct_user_genre.sql') 


execute_sql_script_from_config('SQL_Commands/V7_hook_fct_average_listen_time_per_genre.sql')


SQL script executed successfully.
SQL script executed successfully.
SQL script executed successfully.
SQL script executed successfully.
SQL script executed successfully.
SQL script executed successfully.


In [34]:

execute_sql_script_from_config('SQL_Commands/V8_hook_create_agg_genre.sql')

execute_sql_script_from_config('SQL_Commands/V8_hook_create_fct_age_distribution_per_genre.sql')

execute_sql_script_from_config('SQL_Commands/V9_hook_create_fct_average_listen_time_per_genre.sql') #

execute_sql_script_from_config('SQL_Commands/V10_hook_create_fct_song_genre.sql') #

execute_sql_script_from_config('SQL_Commands/V11_hook_create_agg_songs_per_genre.sql') #fawte table genre mapping

execute_sql_script_from_config('SQL_Commands/V12_hook_create_agg_user_per_genre.sql') #

execute_sql_script_from_config('SQL_Commands/V13_hook_create_agg_genre.sql')

execute_sql_script_from_config('SQL_Commands/V14_hook_create_agg_release_date.sql')

execute_sql_script_from_config('SQL_Commands/V15_hook_create_agg_playlist.sql')

SQL script executed successfully.
SQL script executed successfully.
Error executing SQL script: function avg(text) does not exist
LINE 13:     AVG(music_time_slot) AS average_listen_time
             ^
HINT:  No function matches the given name and argument types. You might need to add explicit type casts.

Error executing SQL script: column "id" does not exist
LINE 12:     id,
             ^
HINT:  There is a column named "id" in table "fct_song_genre", but it cannot be referenced from this part of the query.

Error executing SQL script: column s.genre does not exist
LINE 19:     musicschema.genre_mapping gm ON s.genre = gm.genre_name_...
                                             ^

Error executing SQL script: column "number_of_songs" of relation "aggregate_users_per_genre" does not exist
LINE 10:     (genre_id, genre_name, number_of_songs)
                                    ^

SQL script executed successfully.
SQL script executed successfully.
SQL script executed successfully.
